# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [70]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [72]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,viedma,-40.8135,-62.9967,30.53,33,55,7.21,AR,1681059657
1,1,ribeira grande,38.5167,-28.7000,19.22,82,40,5.66,PT,1681059574
2,2,bredasdorp,-34.5322,20.0403,15.64,61,12,5.91,ZA,1681059578
3,3,watertown,42.3709,-71.1828,12.36,26,0,2.68,US,1681059732
4,4,hamilton,39.1834,-84.5333,17.64,32,0,4.02,US,1681059642


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [73]:
%%capture --no-display

# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)


# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [74]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(abs(city_data_df['Lat']) <= 50) & (city_data_df['Humidity'] <= 80)]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,viedma,-40.8135,-62.9967,30.53,33,55,7.21,AR,1681059657
2,2,bredasdorp,-34.5322,20.0403,15.64,61,12,5.91,ZA,1681059578
3,3,watertown,42.3709,-71.1828,12.36,26,0,2.68,US,1681059732
4,4,hamilton,39.1834,-84.5333,17.64,32,0,4.02,US,1681059642
7,7,umm kaddadah,13.6017,26.6876,31.99,8,84,4.48,SD,1681059776
...,...,...,...,...,...,...,...,...,...,...
519,519,bexbach,49.3461,7.2553,14.53,38,0,4.12,DE,1681059843
520,520,malindi,-3.2175,40.1191,27.70,76,63,3.64,KE,1681059843
522,522,aswan,24.0934,32.9070,36.61,21,0,4.12,EG,1681059608
526,526,gra liyia,35.0167,25.6833,16.74,68,30,3.13,GR,1681059844


### Step 3: Create a new DataFrame called `hotel_df`.

In [75]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.reset_index()

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,viedma,AR,-40.8135,-62.9967,33,
1,2,bredasdorp,ZA,-34.5322,20.0403,61,
2,3,watertown,US,42.3709,-71.1828,26,
3,4,hamilton,US,39.1834,-84.5333,32,
4,7,umm kaddadah,SD,13.6017,26.6876,8,
...,...,...,...,...,...,...,...
300,519,bexbach,DE,49.3461,7.2553,38,
301,520,malindi,KE,-3.2175,40.1191,76,
302,522,aswan,EG,24.0934,32.9070,21,
303,526,gra liyia,GR,35.0167,25.6833,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [83]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionary and convert to JSON
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
viedma - nearest hotel: Níjar
bredasdorp - nearest hotel: Victoria Hotel
watertown - nearest hotel: Four Points by Sheraton Boston Newton
hamilton - nearest hotel: North Vista Manor
umm kaddadah - nearest hotel: No hotel found
mayo - nearest hotel: The Westin
kloulklubed - nearest hotel: The Adventures Inn
mandera - nearest hotel: Kornesh Hotel and Lodging
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
saint-georges - nearest hotel: Deyna's City Inn
laguna - nearest hotel: Holiday Inn Express & Suites
hermanus - nearest hotel: Aloe guest house
mandalgovi - nearest hotel: АЛТАНГОВЬ Hotel
hithadhoo - nearest hotel: Pebbles Inn
alice - nearest hotel: No hotel found
hit - nearest hotel: No hotel found
atuona - nearest hotel: Pearl Resort
roma - nearest hotel: Wingate by Wyndham Rome
miles city - nearest hotel: Historic Olive Hotel
butaritari - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
torbay - nearest hotel: Hampton
jamestown - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
0,viedma,AR,-40.8135,-62.9967,33,Níjar
2,bredasdorp,ZA,-34.5322,20.0403,61,Victoria Hotel
3,watertown,US,42.3709,-71.1828,26,Four Points by Sheraton Boston Newton
4,hamilton,US,39.1834,-84.5333,32,North Vista Manor
7,umm kaddadah,SD,13.6017,26.6876,8,No hotel found
...,...,...,...,...,...,...
519,bexbach,DE,49.3461,7.2553,38,Hotel Klein
520,malindi,KE,-3.2175,40.1191,76,Young Safari
522,aswan,EG,24.0934,32.9070,21,Yaseen hotel
526,gra liyia,GR,35.0167,25.6833,68,Motel gorgona


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [85]:
#reset the index
hotel_df.reset_index()

,index,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,viedma,AR,-40.8135,-62.9967,33,Níjar
1,2,bredasdorp,ZA,-34.5322,20.0403,61,Victoria Hotel
2,3,watertown,US,42.3709,-71.1828,26,Four Points by Sheraton Boston Newton
3,4,hamilton,US,39.1834,-84.5333,32,North Vista Manor
4,7,umm kaddadah,SD,13.6017,26.6876,8,No hotel found
...,...,...,...,...,...,...,...
300,519,bexbach,DE,49.3461,7.2553,38,Hotel Klein
301,520,malindi,KE,-3.2175,40.1191,76,Young Safari
302,522,aswan,EG,24.0934,32.9070,21,Yaseen hotel
303,526,gra liyia,GR,35.0167,25.6833,68,Motel gorgona


In [86]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ['Country','Hotel Name']
)

# Display the map
map_plot_2 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)